# Regressão linear 

Nesta aula faremos uso da base de dados `hotels-vienna` para aprender a fazer uma regressão linear, apresentar seus resultados e interpretá-los.

Como podemos encontrar os hotéis que estão abaixo do preço em relação à distância do centro da cidade? Continuando os estudos de caso anteriores, precisamos descobrir como o preço do hotel está relacionado à distância ao centro da cidade para saber qual preço esperar em quais distâncias. Então, podemos identificar os hotéis com preços mais baixos em comparação com o preço esperado. Começaremos portanto, com uma **regressão linear simples**.


## Importando Pacotes e Baixando os Dados

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
# Carregar os dados
hotels_viena = pd.read_csv("https://osf.io/y6jvb/download")

# 3-4 estrelas, Viena somente, sem valores extremos
df = hotels_viena.loc[
    (hotels_viena["accommodation_type"] == "Hotel")
    & (hotels_viena["city_actual"] == "Vienna")
    & (hotels_viena["stars"] >= 3)
    & (hotels_viena["stars"] <= 4)
    & (hotels_viena["stars"].notna())
    & (hotels_viena["price"] <= 600)
]

In [ ]:
df[['price', 'distance']].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.scatter(df['distance'], df['price'])
ax.set_xlabel("Distância")
ax.set_ylabel("Preço")

## Regressão Linear Simples

Parece haver uma relação negativa entre preço e distância ao centro da cidade. Vamos confirmar esta relação através de uma regressão linear simples e aplicar um teste de hipótese sobre o coeficiente angular estimado. A regressão a ser realizada é:

$\text{preco}_i=\beta_0+\beta_1\text{dist}_i+u_i$,

e testaremos a hipótese nula que $\beta_1=0.$



In [ ]:
regression = smf.ols(formula="price~distance", data=df).fit()

In [ ]:
regression.summary()

É simples interpretar os resultados obtidos desta regressão. Temos um intercepto no valor de 132 e um coeficiente angular associado a distância no valor de -14,41, negativo de acordo com nossas expectativas. O erro padrão deste coeficiente é 2,33 o que nos dá uma estatística t de -6,18. Com esta estatística podemos afirmar que ao nível de significância de 1% a relação entre preço e distância é de fato negativa.

Vamos encontrar as previsões de preços para os hotéis da amostra e plotar a reta de regressão contra o gráfico de dispersão apresentado anteriormente.

In [ ]:
df['previsao'] = regression.predict()

In [ ]:
ax.plot(df['distance'], df['previsao'], color='red')
fig

## Regressão Múltipla

Queremos agora encontrar os hotéis que estão abaixo do preço por sua **qualidade** e **distância** do centro da cidade. Para fazer isso, primeiro precisamos descobrir os preços esperados em vários níveis de distância e qualidade de uma forma que reflita todos os padrões importantes nos dados. Faremos isso através de uma regressão múltipla como esta:

$\text{preco}_i=\beta_0+\beta_1\text{dist}_i+\beta_2\text{qual}_i+u_i$.

Além dos testes t individuais para cada um dos coeficientes, podemos realizar um teste F conjunto para analisar a significância de todos os regressores conjuntamente. 

In [ ]:
mregression = smf.ols('price~distance+rating', data=df).fit()

In [ ]:
mregression.summary()

Enquanto o coeficiente de distância continua estatisticamente significativo e em valor negativo de -11,08, o coeficiente de qualidade (avaliação dos usuários) é positivo e também significativo.

A interpretação de uma regressão múltipla exige o conceito de _ceteris paribus_ (tudo o mais constante). Ou seja, o ganho de 1 estrela nos ratings do hotel significa, em média, um incremento de preço de $39,73, **mantidas as demais variáveis constantes**. Neste caso temos somente outra variável que é a distância.

Já o teste F conjunto tem uma estatística no valor de 39,29. Poderíamos ir numa tabela F com o nível de significância desejado, encontrar a posição correspondente aos 2 graus de liberdade do numerador e 204 do denominador e comparar este valor crítico contra a estatística F. De outra forma, podemos avaliar o p-valor dado de 3,69e-15 (basicamente zero) e então decidir pela rejeição da hipótese nula para qualquer nível de significância tipicamente utilizado. Ou seja, o modelo estimado é informativo. Um ou ambos os regressores nos trazem informações relevantes para prever o preço dos hotéis.

In [ ]:
from statsmodels.stats.anova import anova_lm
anova_lm(mregression, typ=1)